[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/xiptos/is_notes/blob/main/deep_learning.ipynb)

# Deep Learning

## Vectorization examples

Let's start by importing the required libraries

In [ ]:
#importing Libraries
import seaborn as sns
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

For flexibility, we can generate a random classification dataset, with two features and 100 examples.

In [ ]:
from sklearn.datasets import make_classification
from sklearn.datasets import make_blobs
from sklearn.datasets import make_gaussian_quantiles

plt.figure(figsize=(8, 8))

plt.title("One informative feature, one cluster per class", fontsize="small")
X1, Y1 = make_classification(
    n_features=2, n_redundant=0, n_informative=1, n_clusters_per_class=1
)
plt.scatter(X1[:, 0], X1[:, 1], marker="o", c=Y1, s=25, edgecolor="k")


plt.show()

Let's confirm the shape of the examples and the labels.

In [ ]:
print(X1.shape)
print(Y1.shape)
print(X1)
print(Y1)

To be able to use a PyTorch model, it is necessary to convert the numpy arrays to torch tensors.

In [ ]:
# Creating Tensors
X_train=torch.FloatTensor(X1)
y_train=torch.LongTensor(Y1)
print(X_train.shape)
print(y_train.shape)

## Building and training the model

In [ ]:
#Creating the Model
class ANN_model(nn.Module):
    def __init__(self,input_features=2,hidden1=20, hidden2=10,out_features=2):
        super().__init__()
        self.f_connected1 = nn.Linear(input_features,hidden1)
        self.f_connected2 = nn.Linear(hidden1,hidden2)
        self.out = nn.Linear(hidden2,out_features)
        
    def forward(self,x):
        x = torch.sigmoid(self.f_connected1(x))
        x = torch.sigmoid(self.f_connected2(x))
        x = self.out(x)
        return x

In [ ]:
epochs=500


torch.manual_seed(20)
model = ANN_model()
model.parameters

We have to define the loss function as well as the optimizer.

In [ ]:
# Backward Propergation - loss and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

Now we can proceed with the training.

The key concept here is the batch size... think about it and experiment with different values.

In [ ]:
final_losses=[]
batch_size=1
for epoch in range(epochs):

    # X is a torch Variable
    permutation = torch.randperm(X_train.size()[0])

    for i in range(0,X_train.size()[0], batch_size):
        optimizer.zero_grad()

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train[indices], y_train[indices]

        # in case you wanted a semi-full example
        if epoch==0 and i==0:
            print("===> " + str(batch_x.shape))

        outputs = model.forward(batch_x)
        loss = loss_function(outputs,batch_y)
        final_losses.append(loss)
        if epoch % 100 == 1:
            print("Epoch number: {} and the loss : {}".format(i,loss.item()))


        loss.backward()
        optimizer.step()

In [ ]:
#plot the loss function
final_losses_np = []
for l in final_losses:
    final_losses_np.append(l.detach().numpy())

plt.plot(range(int(epochs*100/batch_size)),final_losses_np)
plt.ylabel('Loss')
plt.ylabel('Epochs')